In [ ]:
!pip install --no-index h5py

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
script_dir = os.path.dirname(os.getcwd())  # Get the directory where the script is located
parent_dir = os.path.dirname(script_dir)  # Get the parent directory

print(f"{script_dir = }")
print(f"{parent_dir = }")

sys.path.append(parent_dir)

script_dir = '/lustre06/project/6067616/soroush1/WM_age_of_ultron/notebooks'
parent_dir = '/lustre06/project/6067616/soroush1/WM_age_of_ultron'


In [49]:
from lit_modules.datamodule import MuriDataModule
from argparse import Namespace
from torchvision import models
from torchvision.models import resnet50
import torch as ch
import numpy as np

import torch
import torch.nn.functional as F
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor
import re
from tqdm import tqdm
import h5py
from lit_modules.modules.utils import InferioTemporalLayer
from torch.utils.data import Dataset
import PIL.Image
from torchvision import transforms
from torch.utils.data import DataLoader

# Init Dataset

In [7]:
# Define hyperparameters
hparams = Namespace(
    data_dir="/scratch/soroush1/memorability/muri1320",
    image_size=224,
    batch_size=128,
    num_workers=4,
    change_labels = False,
    pin_memories=[False, False, False],  # [train, val, test]
    return_paths = True
)

# Create the DataModule
data_module = MuriDataModule(hparams)

# Prepare data and setup
data_module.prepare_data()
data_module.setup("test")

# Print dataset sizes
print(f"test dataset size: {len(data_module.test_dataset)}")

test_dl = data_module.test_dataloader()

Muri data not found. Please download the dataset manually.
Test dataset size: 1320
test dataset size: 1320


In [59]:
def extract_number(filename):
    # Extract the number from the filename
    match = re.search(r'im(\d+)\.png', filename)
    if match:
        return int(match.group(1))
    return 0  # Return 0 if no number is found

def sort_filenames(filenames):
    # Sort the filenames based on the extracted number
    return sorted(filenames, key=extract_number)

class MuriDataset(Dataset):
    def __init__(self, root: str, transforms=None, return_paths=False):

        self.root = root

        self.transforms = transforms
        self.img_path_list = sort_filenames(os.listdir(root))
        self.return_path = return_paths

    def __len__(self):
        return len(self.img_path_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root, self.img_path_list[idx])
        # print(f"{img_name = }")
        image = PIL.Image.open(img_name).convert("RGB")

        if self.transforms:
            image = self.transforms(image)

        if self.return_path:
            return image, os.path.basename(img_name)

        return image, img_name

root = "/scratch/soroush1/memorability/muri1320"
input_size = 224
normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )

t = transforms.Compose(
            [
                transforms.Resize(input_size),
                transforms.ToTensor(),
                normalize,
            ]
        )


ds = MuriDataset(root=root, transforms=t)
dl = DataLoader(
            ds,
            batch_size=32,
            shuffle=False,
            num_workers=2,
            pin_memory=True,
        )

for images, names in dl:
    for image, name in zip(images, names):
        print(f"{os.path.basename(name)}: {img.size()}")
    break

im0000.png: torch.Size([32, 3, 224, 224])
im0001.png: torch.Size([32, 3, 224, 224])
im0002.png: torch.Size([32, 3, 224, 224])
im0003.png: torch.Size([32, 3, 224, 224])
im0004.png: torch.Size([32, 3, 224, 224])
im0005.png: torch.Size([32, 3, 224, 224])
im0006.png: torch.Size([32, 3, 224, 224])
im0007.png: torch.Size([32, 3, 224, 224])
im0008.png: torch.Size([32, 3, 224, 224])
im0009.png: torch.Size([32, 3, 224, 224])
im0010.png: torch.Size([32, 3, 224, 224])
im0011.png: torch.Size([32, 3, 224, 224])
im0012.png: torch.Size([32, 3, 224, 224])
im0013.png: torch.Size([32, 3, 224, 224])
im0014.png: torch.Size([32, 3, 224, 224])
im0015.png: torch.Size([32, 3, 224, 224])
im0016.png: torch.Size([32, 3, 224, 224])
im0017.png: torch.Size([32, 3, 224, 224])
im0018.png: torch.Size([32, 3, 224, 224])
im0019.png: torch.Size([32, 3, 224, 224])
im0020.png: torch.Size([32, 3, 224, 224])
im0021.png: torch.Size([32, 3, 224, 224])
im0022.png: torch.Size([32, 3, 224, 224])
im0023.png: torch.Size([32, 3, 224

# Init Model

In [60]:
class BlurPoolConv2d(ch.nn.Module):

    # Purpose: This class creates a convolutional layer that first applies a blurring filter to the input before performing the convolution operation.
    # Condition: The function apply_blurpool iterates over all layers of the model and replaces convolution layers (ch.nn.Conv2d) with BlurPoolConv2d if they have a stride greater than 1 and at least 16 input channels.
    # Preventing Aliasing: Blurring the output of convolution layers (especially those with strides greater than 1) helps to reduce aliasing effects. Aliasing occurs when high-frequency signals are sampled too sparsely, leading to incorrect representations.
    # Smooth Transitions: Applying a blur before downsampling ensures that transitions between pixels are smooth, preserving important information in the feature maps.
    # Stabilizing Training: Blurring can help stabilize training by reducing high-frequency noise, making the model less sensitive to small changes in the input data.
    def __init__(self, conv):
        super().__init__()
        default_filter = ch.tensor([[[[1, 2, 1], [2, 4, 2], [1, 2, 1]]]]) / 16.0
        filt = default_filter.repeat(conv.in_channels, 1, 1, 1)
        self.conv = conv
        self.register_buffer("blur_filter", filt)

    def forward(self, x):
        blurred = F.conv2d(
            x,
            self.blur_filter,
            stride=1,
            padding=(1, 1),
            groups=self.conv.in_channels,
            bias=None,
        )
        return self.conv.forward(blurred)


def remove_prefix(state_dict, prefix):
    """
    Remove a prefix from the state_dict keys.

    Args:
    state_dict (dict): State dictionary from which the prefix will be removed.
    prefix (str): Prefix to be removed.

    Returns:
    dict: State dictionary with prefix removed from keys.
    """
    return {
        key[len(prefix) :]: value
        for key, value in state_dict.items()
        if key.startswith(prefix)
    }


def match_and_load_weights(checkpoint_state_dict, model, prefix="module."):
    """
    Match weights from checkpoint_state_dict with model's state_dict and load them into the model.

    Args:
    checkpoint_state_dict (dict): State dictionary from checkpoint.
    model (torch.nn.Module): The model instance.
    prefix (str): Prefix to be removed from checkpoint keys.

    Returns:
    None
    """
    # Remove the prefix from checkpoint state dict keys
    cleaned_checkpoint_state_dict = remove_prefix(checkpoint_state_dict, prefix)

    model_state_dict = model.state_dict()
    matched_weights = {}

    # Iterate over the cleaned checkpoint state dict
    for ckpt_key, ckpt_weight in cleaned_checkpoint_state_dict.items():
        if ckpt_key in model_state_dict:
            # If the layer name matches, add to the matched_weights dict
            matched_weights[ckpt_key] = ckpt_weight
        else:
            print(
                f"Layer {ckpt_key} from checkpoint not found in the model state dict."
            )

    return matched_weights

def create_feature_extractor_model(config):
    arch = config["arch"]
    weights = config["weights"]
    use_blurpool = config["use_blurpool"]
    layer_name = config["layer_name"]
    device = ch.device("cuda" if ch.cuda.is_available() else "cpu")
    print(f"{device = }")

    model = getattr(models, arch)(weights=None)
    

    def apply_blurpool(mod: ch.nn.Module):
        for name, child in mod.named_children():
            if isinstance(child, ch.nn.Conv2d) and (
                np.max(child.stride) > 1 and child.in_channels >= 16
            ):
                setattr(mod, name, BlurPoolConv2d(child))
            else:
                apply_blurpool(child)

    if use_blurpool:
        apply_blurpool(model)

    ckpt = ch.load(weights,weights_only=True, map_location=device)
    print(f"{list(ckpt.keys())[:10] = }")
    ckpt = match_and_load_weights(ckpt, model)
    print(f"{list(ckpt.keys())[:10] = }")
    
    
    model = model.to(device)    
    model.load_state_dict(ckpt)
    model.eval()
    model = create_feature_extractor(model=model, return_nodes=layer_name)
    
    return model

config = {"weights": "/home/soroush1/projects/def-kohitij/soroush1/idiosyncrasy/notebooks/experiments/weights/experiment_different_initialization/resnet50-0/final_weights.pt", "arch": "resnet50", "use_blurpool": True, "layer_name": {f"{InferioTemporalLayer.RESNET50.value}":"IT"}}
model = create_feature_extractor_model(config)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
rand_inp = torch.rand((1, 3, 224, 224))
rand_inp = rand_inp.to(device)
output = model(rand_inp)
output = output["IT"].detach().cpu().numpy()
output = output.reshape(output.shape[0], -1)
output.shape

device = device(type='cuda')
list(ckpt.keys())[:10] = ['module.conv1.weight', 'module.bn1.weight', 'module.bn1.bias', 'module.bn1.running_mean', 'module.bn1.running_var', 'module.bn1.num_batches_tracked', 'module.layer1.0.conv1.weight', 'module.layer1.0.bn1.weight', 'module.layer1.0.bn1.bias', 'module.layer1.0.bn1.running_mean']
list(ckpt.keys())[:10] = ['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean']


(1, 100352)

# Feature Extraction

In [79]:
def sort_batch_by_filename(tensor, filenames):
    # Extract indices from filenames
    indices = [int(re.search(r'(\d+)', fname).group()) for fname in filenames]
    
    # Create a list of (index, tensor_slice, filename) tuples
    indexed_data = list(zip(indices, tensor, filenames))
    
    # Sort the list based on the extracted indices
    sorted_data = sorted(indexed_data, key=lambda x: x[0])
    
    # Unzip the sorted list
    _, sorted_tensor, sorted_filenames = zip(*sorted_data)
    
    # Stack the tensor slices back into a single tensor
    sorted_tensor = torch.stack(sorted_tensor)
    
    return sorted_tensor, sorted_filenames


def save_h5(data, dataset_name, model_name, task_name, dst_path: str, layer_name: str = "it", ):
    """
    Load features from a pickle file and save them to an HDF5 file.

    Args:
    model_name (str): Name of the model used to extract features.
    task_name (str): Name of the task or dataset.
    layer_name (str): Name of the layer from which features were extracted.
    dst_path (str): Directory to save the HDF5 file.

    Returns:
    None
    """

    print(
        f"Model: {model_name}, Task: {task_name}, Layer: {layer_name}, Shape: {data.shape}"
    )

    # Ensure the destination directory exists
    os.makedirs(dst_path, exist_ok=True)

    # Construct the full path for the HDF5 file
    h5_file = os.path.join(dst_path, f"{dataset_name}_{model_name}_{task_name}_{layer_name}.h5")

    # Save the NumPy array as a .h5 file
    with h5py.File(h5_file, "w") as hf:
        hf.create_dataset("features", data=data)

    print(f"Features saved to {h5_file}")

def extract_and_concatenate_features(model, test_dl, device):
    all_outputs = []
    all_img_paths = []
    for img, img_path in tqdm(test_dl):
        with torch.no_grad():  # Disable gradient calculation
            # logger.info(f"Using input size: {img.size()}")
            img = img.to(device)
            output = model(img)["IT"]  # extract IT layer
            output, img_path = sort_batch_by_filename(output, img_path)
        
        all_outputs.append(output.cpu())  # Move to CPU to save GPU memory
        all_img_paths.extend(img_path)
        
        # Delete batch data to free up memory
        del img, output
        torch.cuda.empty_cache()  # Clear CUDA cache
    
    # Concatenate all outputs into a single tensor
    concatenated_output = torch.cat(all_outputs, dim=0)
    
    # Delete all_outputs to free up memory
    del all_outputs
    torch.cuda.empty_cache()
    
    # Reshape the output to (samples, -1)
    reshaped_output = concatenated_output.view(concatenated_output.size(0), -1)
    
    # Convert to numpy and delete the torch tensor
    reshaped_output = reshaped_output.detach().numpy()
    del concatenated_output
    torch.cuda.empty_cache()
    
    return reshaped_output, all_img_paths

In [80]:
# Inputs
total_run = 12
for run_id in tqdm(range(0, total_run + 1), desc="different_run"):
    config = {"weights": f"/home/soroush1/projects/def-kohitij/soroush1/idiosyncrasy/notebooks/experiments/weights/experiment_different_initialization/resnet50-{run_id}/final_weights.pt", "arch": "resnet50", "use_blurpool": True, "layer_name": {f"{InferioTemporalLayer.RESNET50.value}":"IT"}}
    root = "/scratch/soroush1/memorability/muri1320"
    input_size = 224
    normalize = transforms.Normalize(
                mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
            )
    
    t = transforms.Compose(
                [
                    transforms.Resize(input_size),
                    transforms.ToTensor(),
                    normalize,
                ]
            )
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    
    # Model
    model = create_feature_extractor_model(config)
    
    # Dataset
    ds = MuriDataset(root=root, transforms=t)
    dl = DataLoader(
                ds,
                batch_size=32,
                shuffle=False,
                num_workers=2,
                pin_memory=True,
    
            )
    
    # Extracting Feature
    concatenated_features, all_image_paths = extract_and_concatenate_features(model, dl, device)
    print(f"Concatenated features shape: {concatenated_features.shape}")
    print(f"Total number of images: {len(all_image_paths)}")
    
    dataset_name = "muri1320"
    model_name = "resnet50"
    task_name = f"diffInitRes{run_id}"
    layer_name = "IT"
    
    dst_path = "./features"
    
    # Call the save_h5 function
    save_h5(concatenated_features, dataset_name=dataset_name, model_name=model_name, task_name=task_name, dst_path=dst_dir, layer_name=layer_name)
    
    # Load the saved data and compare
    expected_file = os.path.join(dst_path, f"{dataset_name}_{model_name}_{task_name}_{layer_name}.h5")
    with h5py.File(expected_file, "r") as hf:
        saved_data = hf["features"][:]
    
    print(saved_data.shape)

different_run:   0%|          | 0/13 [00:00<?, ?it/s]

device = device(type='cuda')
list(ckpt.keys())[:10] = ['module.conv1.weight', 'module.bn1.weight', 'module.bn1.bias', 'module.bn1.running_mean', 'module.bn1.running_var', 'module.bn1.num_batches_tracked', 'module.layer1.0.conv1.weight', 'module.layer1.0.bn1.weight', 'module.layer1.0.bn1.bias', 'module.layer1.0.bn1.running_mean']
list(ckpt.keys())[:10] = ['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean']



  7%|▋         | 3/42 [00:00<00:03, 11.72it/s]

['/scratch/soroush1/memorability/muri1320/im0000.png', '/scratch/soroush1/memorability/muri1320/im0001.png', '/scratch/soroush1/memorability/muri1320/im0002.png', '/scratch/soroush1/memorability/muri1320/im0003.png', '/scratch/soroush1/memorability/muri1320/im0004.png', '/scratch/soroush1/memorability/muri1320/im0005.png', '/scratch/soroush1/memorability/muri1320/im0006.png', '/scratch/soroush1/memorability/muri1320/im0007.png', '/scratch/soroush1/memorability/muri1320/im0008.png', '/scratch/soroush1/memorability/muri1320/im0009.png']
['/scratch/soroush1/memorability/muri1320/im0032.png', '/scratch/soroush1/memorability/muri1320/im0033.png', '/scratch/soroush1/memorability/muri1320/im0034.png', '/scratch/soroush1/memorability/muri1320/im0035.png', '/scratch/soroush1/memorability/muri1320/im0036.png', '/scratch/soroush1/memorability/muri1320/im0037.png', '/scratch/soroush1/memorability/muri1320/im0038.png', '/scratch/soroush1/memorability/muri1320/im0039.png', '/scratch/soroush1/memorab


 21%|██▏       | 9/42 [00:00<00:01, 19.69it/s]

...

['/scratch/soroush1/memorability/muri1320/im1056.png', '/scratch/soroush1/memorability/muri1320/im1057.png', '/scratch/soroush1/memorability/muri1320/im1058.png', '/scratch/soroush1/memorability/muri1320/im1059.png', '/scratch/soroush1/memorability/muri1320/im1060.png', '/scratch/soroush1/memorability/muri1320/im1061.png', '/scratch/soroush1/memorability/muri1320/im1062.png', '/scratch/soroush1/memorability/muri1320/im1063.png', '/scratch/soroush1/memorability/muri1320/im1064.png', '/scratch/soroush1/memorability/muri1320/im1065.png']
['/scratch/soroush1/memorability/muri1320/im1088.png', '/scratch/soroush1/memorability/muri1320/im1089.png', '/scratch/soroush1/memorability/muri1320/im1090.png', '/scratch/soroush1/memorability/muri1320/im1091.png', '/scratch/soroush1/memorability/muri1320/im1092.png', '/scratch/soroush1/memorability/muri1320/im1093.png', '/scratch/soroush1/memorability/muri1320/im1094.png', '/scratch/soroush1/memorability/muri1320/im1095.png', '/scratch/soroush1/memorab


100%|██████████| 42/42 [00:01<00:00, 21.75it/s]

['/scratch/soroush1/memorability/muri1320/im1248.png', '/scratch/soroush1/memorability/muri1320/im1249.png', '/scratch/soroush1/memorability/muri1320/im1250.png', '/scratch/soroush1/memorability/muri1320/im1251.png', '/scratch/soroush1/memorability/muri1320/im1252.png', '/scratch/soroush1/memorability/muri1320/im1253.png', '/scratch/soroush1/memorability/muri1320/im1254.png', '/scratch/soroush1/memorability/muri1320/im1255.png', '/scratch/soroush1/memorability/muri1320/im1256.png', '/scratch/soroush1/memorability/muri1320/im1257.png']
['/scratch/soroush1/memorability/muri1320/im1280.png', '/scratch/soroush1/memorability/muri1320/im1281.png', '/scratch/soroush1/memorability/muri1320/im1282.png', '/scratch/soroush1/memorability/muri1320/im1283.png', '/scratch/soroush1/memorability/muri1320/im1284.png', '/scratch/soroush1/memorability/muri1320/im1285.png', '/scratch/soroush1/memorability/muri1320/im1286.png', '/scratch/soroush1/memorability/muri1320/im1287.png', '/scratch/soroush1/memorab


different_run: 100%|██████████| 13/13 [00:39<00:00,  3.03s/it]

Features saved to ./features/muri1320_resnet50_diffInitRes12_IT.h5
(1320, 100352)
